In [ ]:
import geopandas as gpd
import os
import psycopg2

In [ ]:
def open_connection():
    conn = \
        psycopg2.connect( \
            host = 'localhost',
            database = 'queroponto',
            user = 'postgres',
            password = 'postgres')
    
    conn.autocommit = True
    cur = conn.cursor()
    
    return conn, cur

In [ ]:
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [ ]:
def insert_data(cur, values):
    query = f" \
        INSERT INTO public.setores_censitarios (cod_ibge, id_cidade, geom) VALUES \
        ({values[0]}, {values[1]}, ST_GeomFromText('{values[2]}', 4326));";

    try:
        cur.execute(query)
    except Exception as e:
        print(f"Failed to insert: {query}")
        print(e)

---

In [ ]:
path = '../../arquivos/ibge/setores-censitarios'

In [ ]:
folders = os.listdir(path)

In [ ]:
conn, cur = open_connection()

In [ ]:
for folder in folders:
    for file in os.listdir(f"{path}/{folder}"):
        if file.split('.')[-1] == 'shp':
            gdf = gpd.read_file(f"{path}/{folder}/{file}")
            
            for row in gdf.itertuples():
                try:
                    values = [int(row[2]), int(row[2][:7]), row[-1]]
                    insert_data(cur, values)
                except Exception as e:
                    print(f"Failed to insert: {folder}/{file}/{row}")
                    print(e)

In [ ]:
close_connection(conn, cur)